# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846773104028                   -0.70    4.5
  2   -7.852309274213       -2.26       -1.53    1.0
  3   -7.852646228748       -3.47       -2.52    3.2
  4   -7.852646676854       -6.35       -3.36    2.0
  5   -7.852646685742       -8.05       -4.73    1.2
  6   -7.852646686722       -9.01       -5.20    3.5
  7   -7.852646686730      -11.11       -6.28    1.8
  8   -7.852646686730      -12.71       -7.56    2.2
  9   -7.852646686730      -15.05       -7.60    2.2
 10   -7.852646686730   +    -Inf       -8.83    2.0
 11   -7.852646686730      -15.05       -9.58    2.5
 12   -7.852646686730      -14.75      -10.68    2.0
 13   -7.852646686730   +  -15.05      -10.13    1.0
 14   -7.852646686730   +  -14.45      -10.05    1.0
 15   -7.852646686730      -14.45      -10.63    1.0
 16   -7.852646686730   +  -14.75       -9.82    1.5
 17   -7.852646686730      -15.05      -10.50 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846813057554                   -0.70           4.5
  2   -7.852528898657       -2.24       -1.63   0.80    2.2
  3   -7.852637010409       -3.97       -2.71   0.80    1.0
  4   -7.852646479642       -5.02       -3.29   0.80    2.2
  5   -7.852646670261       -6.72       -4.16   0.80    1.2
  6   -7.852646686397       -7.79       -4.74   0.80    2.0
  7   -7.852646686710       -9.50       -5.42   0.80    1.5
  8   -7.852646686729      -10.72       -6.45   0.80    1.8
  9   -7.852646686730      -12.36       -7.56   0.80    2.5
 10   -7.852646686730      -14.01       -7.98   0.80    2.2
 11   -7.852646686730      -14.75       -8.45   0.80    1.0
 12   -7.852646686730   +    -Inf       -9.19   0.80    1.8
 13   -7.852646686730   +    -Inf       -9.72   0.80    1.8
 14   -7.852646686730   +  -15.05      -10.53   0.80    1.5
 15   -7.852646686730   +  -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.362744e+01     3.295891e+00
 * time: 0.44206690788269043
     1     1.217732e+00     1.760908e+00
 * time: 0.7732341289520264
     2    -1.705896e+00     2.058684e+00
 * time: 0.7976200580596924
     3    -3.749497e+00     1.695687e+00
 * time: 0.83489990234375
     4    -5.051752e+00     1.571388e+00
 * time: 0.871406078338623
     5    -6.739739e+00     7.838982e-01
 * time: 0.9067039489746094
     6    -7.378367e+00     4.000450e-01
 * time: 0.9420299530029297
     7    -7.628567e+00     1.841267e-01
 * time: 0.9671890735626221
     8    -7.745297e+00     1.158328e-01
 * time: 0.9920430183410645
     9    -7.804132e+00     9.759189e-02
 * time: 1.017798900604248
    10    -7.827757e+00     6.481818e-02
 * time: 1.042815923690796
    11    -7.841549e+00     6.482859e-02
 * time: 1.0682499408721924
    12    -7.847314e+00     4.246039e-02
 * time: 1.0941789150238037
    13    -7.850393e+00     3.034231e-02
 * time: 1.11964011192321

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846829184847                   -0.70    4.5
  2   -7.852317021731       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686712                   -2.54
  2   -7.852646686730      -10.74       -6.00
  3   -7.852646686730      -15.05      -12.69


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.791781588506487e-14
|ρ_newton - ρ_scfv| = 1.5324629805970736e-12
|ρ_newton - ρ_dm|   = 1.6629163527863876e-10
